#### CRAWLING   카테고리에서 장르만 / top100 월간/주간?  장르별 링크
- 01_로맨스 ~ 1833  top100  주간 > https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=201

- 02_로판 ~ 619  top100  주간 > https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=207

- 03_판타지 ~493   top100  주간 > https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=202

- 04_현판 ~401  top100  주간 > https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=208

- 05_무협 ~218  top100  주간 > https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=206

- n06_미스터리 ~288  top100  월간  8편 > https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=203
- 06_미스터리 ~288  장르 평점순  12편 https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203&orderTypeCode=star_score&is&isFinished=false

- 07_라이트노벨 ~39  top100  월간  11편 > https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=205
- 07_라이트노벨 ~39  장르 평점순  9편 > https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=star_score&is&isFinished=false

- 08_BL ~401   top100  주간 > https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=209

In [2]:
import urllib.request
from bs4 import BeautifulSoup
import json
import os
from urllib.parse import urljoin
from datetime import datetime

# 📌 크롤링할 URL 리스트
url_list = [
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=201",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=207",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=202",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=208",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=206",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=203",
    "https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203&orderTypeCode=star_score&is&isFinished=false",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=MONTHLY&categoryCode=205",
    "https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=star_score&is&isFinished=false",
    "https://series.naver.com/novel/top100List.series?rankingTypeCode=WEEKLY&categoryCode=209",
]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 📌 HTML 가져오기
def fetch_html(url):
    try:
        req = urllib.request.Request(url, headers=headers)
        with urllib.request.urlopen(req) as response:
            return response.read()
    except Exception as e:
        print(f"❌ [오류] {url} 크롤링 실패: {e}")
        return None

# 📌 소설 URL 크롤링 함수 (Top 리스트 크롤링 실패 시 평점순 크롤링)
def get_novel_urls(url):
    sourcecode = fetch_html(url)
    if sourcecode is None:
        return []  # 크롤링 실패 시 빈 리스트 반환

    soup = BeautifulSoup(sourcecode, "html.parser")
    novel_urls = set()  # 중복 방지

    # ✅ 1차 크롤링: 주간 Top 리스트
    for li in soup.select("div.comic_lst_thum_wrap ul.comic_top_lst li"):
        link = li.find("a", class_="pic")
        if link and "href" in link.attrs:
            full_url = urljoin("https://series.naver.com", link["href"])
            novel_urls.add(full_url)

    # 🔄 만약 주간 Top 리스트 크롤링이 실패했다면? 👉 평점순 크롤링 실행
    if not novel_urls:
        print(f"⚠️ {url} → Top 리스트 크롤링 실패 → 평점순 크롤링 실행")
        for li in soup.select("div.lst_thum_wrap ul.lst_list li"):
            link = li.find("a", class_="pic")
            if link and "href" in link.attrs:
                href_value = link["href"]
                if "categoryProductList.series" not in href_value:
                    full_url = urljoin("https://series.naver.com", href_value)
                    novel_urls.add(full_url)

    return list(novel_urls)

# 📌 JSON 저장 함수
def save_json(path: str, file_name: str, data: list):
    os.makedirs(path, exist_ok=True)
    with open(os.path.join(path, file_name), "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# ✅ 실행
if __name__ == "__main__":
    all_novel_urls = set()  # 전체 크롤링된 URL 저장 (중복 방지)

    for url in url_list:
        novel_urls = get_novel_urls(url)
        all_novel_urls.update(novel_urls)  # 새로운 URL 추가

    # 저장 경로 생성
    DATA_DIR = "sample"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # 파일명 자동 변경
    file_name = f"wn_ns_sample_{timestamp}.json"

    # JSON 파일 저장
    save_json(DATA_DIR, file_name, list(all_novel_urls))

    # 결과 출력
    print(f"✅ 총 {len(all_novel_urls)}개의 소설 URL이 저장되었습니다.")
    print(f"📁 JSON 파일 경로: {DATA_DIR}/{file_name}")

⚠️ https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=203&orderTypeCode=star_score&is&isFinished=false → Top 리스트 크롤링 실패 → 평점순 크롤링 실행
⚠️ https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=star_score&is&isFinished=false → Top 리스트 크롤링 실패 → 평점순 크롤링 실행
✅ 총 185개의 소설 URL이 저장되었습니다.
📁 JSON 파일 경로: sample/wn_ns_sample_20250217_143846.json


#### 페이지 요소 추출  (미완성)

In [ ]:

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import lxml
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import urlparse, urlunparse

from dotenv import load_dotenv
import pickle
import json
import time
import os

# Load environment variables
load_dotenv()
naver_id = os.getenv("NAVER_ID")
naver_pw = os.getenv("NAVER_PWD")

def save_cookies(driver, path):
    with open(path, "wb") as file:
        pickle.dump(driver.get_cookies(), file)

def load_cookies(driver, path):
    if os.path.exists(path):
        with open(path, "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)

def login_naver(driver):
    driver.get("https://nid.naver.com/nidlogin.login")
    time.sleep(2)  # 페이지 로딩 대기

    driver.execute_script("document.getElementById('id').value = arguments[0]", naver_id)
    driver.execute_script("document.getElementById('pw').value = arguments[0]", naver_pw)
    driver.find_element(By.ID, "log.login").click()
    time.sleep(5)  # 로그인 완료 대기
    save_cookies(driver, "naver_cookies.pkl")

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def crawling(url, driver):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기
    
    series_novel = {
        "type": "웹소설",
        "platform": "네이버 시리즈"
    }

    
    try:
        title_selector = r"#content > div.end_head > h2"
        title_element = driver.find_element(By.CSS_SELECTOR, title_selector).text
    except:
        title_element = "-"
    series_novel["title"] = title_element


    try:
        status_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(1) > span"
        status_element = driver.find_element(By.CSS_SELECTOR, status_selector).text
    except:
        status_element = "-"
    series_novel["status"] = status_element
    

    try:
        thumbnail_selector = r"#container > div.aside.NE\\=a\\:nvi > span.pic_area > img"
        thumbnail_element = driver.find_element(By.CSS_SELECTOR, thumbnail_selector).get_attribute("src")
    except:
        try:
            thumbnail_selector = r'//*[@id="container"]/div[1]/a/img'
            thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
        except:
            try:
                thumbnail_selector = r'//*[@id="container"]/div[1]/span/img'
                thumbnail_element = driver.find_element(By.XPATH, thumbnail_selector).get_attribute("src")
            except:
                thumbnail_element = "-"
    series_novel["thumbnail"] = thumbnail_element


    try:
        genre_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(2) > span > a"
        genre_element = driver.find_element(By.CSS_SELECTOR, genre_selector).text
    except:
        genre_element = "-"
    series_novel["genre"] = genre_element


    try:
        views_selector = r"-"
        views_element = driver.find_element(By.CSS_SELECTOR, views_selector).text
    except:
        views_element = "-"
    series_novel["views"] = views_element


    try:
        rating_selector = r"#content > div.end_head > div.score_area > em"
        rating_element = driver.find_element(By.CSS_SELECTOR, rating_selector).text
        rating_element = float(rating_element)
    except:
        rating_element = "-"
    series_novel["rating"] = rating_element


    try:
        like_selector = r"#content > div.end_head > div.user_action_area > ul > li:nth-child(2) > div > a > em"
        like_element = driver.find_element(By.CSS_SELECTOR, like_selector).text
        like_element = int(like_element.replace(",", ""))   
    except:
        like_element = "-"
    series_novel["like"] = like_element


    try:
        CLICKER_SELECTOR = "#content > div.end_dsc > div:nth-child(1) > span > a"
        driver.find_element(By.CSS_SELECTOR, CLICKER_SELECTOR).click()
        synopsis_selector = "#content > div.end_dsc.open > div:nth-child(2)"
        synopsis_element = driver.find_element(By.CSS_SELECTOR, synopsis_selector).text
    except:
        try:
            synopsis_selector = r'//*[@id="content"]/div[2]/div'
            synopsis_element = driver.find_element(By.XPATH, synopsis_selector).text
        except:
            synopsis_element = "-"
    series_novel["synopsis"] = synopsis_element

    try:
        keywords_selector = r"-"
        keywords_element = driver.find_element(By.CSS_SELECTOR, keywords_selector).text
        keywords_element = keywords_element.replace("#", "")
        keywords_element = keywords_element.split("\n")
    except:
        keywords_element = "-"
    series_novel["keywords"] = keywords_element


    try:
        author_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(3) > a"
        author_element = driver.find_element(By.CSS_SELECTOR, author_selector).text
    except:
        author_element = "-"
    series_novel["author"] = author_element


    try:
        illustrator_selector = r"-"
        illustrator_element = driver.find_element(By.CSS_SELECTOR, illustrator_selector).text
    except:
        illustrator_element = "-"
    series_novel["illustrator"] = illustrator_element


    try:                     
        age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
        age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
    except:
        try:
            age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(6)"
            age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
            exclude_keywords = ["그림", "출판사"]
            if not any(keyword in  age_rating_element for keyword in exclude_keywords):
                age_rating_element = int(age_rating_element)
        except:
            try:
                age_rating_selector = r"#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(5)"
                age_rating_element = driver.find_element(By.CSS_SELECTOR, age_rating_selector).text
                exclude_keywords = ["그림", "출판사"]
                if not any(keyword in  age_rating_element for keyword in exclude_keywords):
                    age_rating_element = int(age_rating_element)
                else:
                    age_rating_element = "-"
            except:
                try:
                    age_rating_selector = r'//*[@id="content"]/ul[1]/li/ul/li[5]' 
                    age_rating_element = driver.find_element(By.XPATH, age_rating_selector).text
                    exclude_keywords = ["그림", "출판사"]
                except:
                    age_rating_element = "-"
    series_novel["age_rating"] = age_rating_element

    free_selector = r"#content > div.end_title_fixed_price_sign.NE\=a\:nvi > div > div.area_text_information > strong"
    wait_selector = r"#container > div.aside.NE\=a\:nvi > span > em.ico_end_head.daily_free2"
    price_selector = r"#content > div.area_ebook_price_information > div > dl > dd > div > div.area_price"


    try:
        driver.find_element(By.CSS_SELECTOR, free_selector)
        price_element = driver.find_element(By.CSS_SELECTOR, free_selector).text.get_attribute("무료")
    except:
        try:
            driver.find_element(By.CSS_SELECTOR, wait_selector)
            price_element = driver.find_element(By.CSS_SELECTOR, wait_selector).text
        except:
            try:
                price_element = driver.find_element(By.CSS_SELECTOR, price_selector).text
            except:
                price_element = "-"
    series_novel["price"] = price_element
    

    try:
        episode_selector = r"#content > h5 > strong"
        episode_element = driver.find_element(By.CSS_SELECTOR, episode_selector).text
        episode_element["episode"] = int(episode_element)
    except:
        try:
            episode_selector = "#content > h5.end_total_episode > strong"
            episode_element = driver.find_element(By.CSS_SELECTOR, episode_selector).text
            episode_element = int(episode_element)
        except:
            episode_element = "-"
    series_novel["episode"] = episode_element
    
    URL = url + "?tab_type=overview"
    driver.get(URL)
    series_novel["url"] = url
    
    return series_novel

if __name__ == "__main__":
    driver = initialize_driver()
    
    # 쿠키 로드 시도
    driver.get("https://www.naver.com")
    load_cookies(driver, "naver_cookies.pkl")
    driver.get("https://nid.naver.com/nidlogin.login")
    time.sleep(2)
    
    if "nid.naver.com" in driver.current_url:  # 로그인 유지가 안 된 경우 로그인 실행
        login_naver(driver)
    
    url_list = []
    result = crawling(url, driver)
    
    driver.quit()
    
    # 크롤링 결과 저장
    with open("crawling_result.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
    
    print("크롤링 결과 저장 완료!")
